In [1]:
import json
import stackprinter  # type: ignore
import jupyter_black  # type: ignore
from dotenv import load_dotenv  # type: ignore
from typing import Literal
from baml_client.async_client import b

from baml_agents import ActionRunner
from baml_agents import JupyterBamlMonitor
from baml_agents import init_logging, with_model, Action, Result
from baml_client import types
from notebooks._utils import (
    celsius_to_fahrenheit,
    city_to_number,
    city_to_weather_condition,
)

init_logging(level="INFO")
# stackprinter.set_excepthook()
load_dotenv()
jupyter_black.load()

In [2]:
b_low_latency = with_model(b, "gpt-4.1-nano")


async def summarize(action, result):
    return await b_low_latency.SummarizeAction(
        action=json.dumps(action.chosen_action, indent=4),
        result=result.content if not result.error else result,
    )


def new_interaction(action, result):
    return types.Interaction(
        action=str(action),
        result=types.Result(content=result.content, error=result.error),
    )


def is_result_available(action) -> str | None:
    if action.chosen_action["action_id"] != Stop.get_action_id():  # type: ignore
        return None
    return action.chosen_action["final_result"]

# Simple BAML Agent demo

Let's put it all together:

In [3]:
class Stop(Action):
    """You're ready to provide the final answer or are unable to continue (e.g. stuck in a loop)."""

    final_result: str

    def run(self) -> Result:
        raise NotImplementedError("Stop action should not be called directly.")


class GetWeatherInfo(Action):
    """Get weather information for a given city."""

    city: str
    measurement: Literal["celsius", "fahrenheit"] | None = None

    def run(self) -> Result:
        self.measurement = self.measurement or "celsius"
        c = city_to_number(self.city, -10, 35)
        condition = city_to_weather_condition(self.city)
        if self.measurement.lower() == "fahrenheit":  # type: ignore
            c, u = celsius_to_fahrenheit(c), "fahrenheit"
        else:
            u = "celsius"
        content = f"The weather in {self.city} is {round(c, 1)} degrees {u} with {condition.lower()} conditions."
        return Result(content=content, error=False)


GetWeatherInfo(city="New York", measurement="fahrenheit").run()

Result(content='The weather in New York is 26.6 degrees fahrenheit with stormy conditions.', error=False)

In [4]:
from baml_client.async_client import b
from baml_client.type_builder import TypeBuilder

b = with_model(b, "gpt-4.1")
r = ActionRunner(TypeBuilder, b=b, cache=True)
r.add_from_mcp_server(server="uvx mcp-server-calculator")
r.add_from_mcp_server(server="uvx mcp-timeserver")  # Note: you can also add URLs
r.add_action(GetWeatherInfo)
r.add_action(Stop)


async def execute_task(r, b, task: str) -> str:
    interactions = []
    while True:
        action = await b.GetNextAction(
            task, interactions, baml_options={"tb": r.tb("NextAction")}
        )

        if result := is_result_available(action):
            return result

        result = r.run(action)

        interactions.append(new_interaction(action, result))
        print(await summarize(action, result))


task = "State the current date along with average temperature between LA, NY and Chicago in Fahrenheit."
await execute_task(r, b, task)

Retrieved the current date and time as of the latest system update.
Retrieved the current weather information for Los Angeles in Fahrenheit.
Fetched current weather details for New York, showing 26.6°F with stormy conditions.
Retrieved the current weather in Chicago: 48.2°F cloudy.
Calculated the average of 73.4, 26.6, and 48.2 (result: 49.4).


'The current date is 2025-04-23. The average temperature between Los Angeles, New York, and Chicago is 49.4°F.'

We got the answer but we'd like more transparency on the what are the exact prompts and completions

# Inspect prompts and completions

In [5]:
task = "State the current date along with average temperature between LA, NY and Chicago in Fahrenheit."

with JupyterBamlMonitor(b) as m:
    result = await execute_task(r, m.b, task)

{'chosen_action': {'action_id': 'stop', 'final_result': 'The current date is 2025-04-23. The average temperature between Los Angeles, New York, and Chicago is 49.4°F.'}}

Retrieved the current date and time as of the latest system update.
Retrieved the current weather information for Los Angeles in Fahrenheit.
Fetched current weather details for New York, showing 26.6°F with stormy conditions.
Retrieved the current weather in Chicago: 48.2°F cloudy.
Calculated the average of 73.4, 26.6, and 48.2 (result: 49.4).


<IPython.core.display.Javascript object>

In [6]:
await m.display_session(task)

![Image](https://github.com/user-attachments/assets/ea55c3e7-147d-41aa-99ce-40e4783f7818)